5. Optimize the CNN Model Architecture
- 5.1 Adding more convolution layers or dropout layers
- 5.2 Hyperparameter fine-tuning
    - 5.2.1 Grid search, smaller kernel size, batch size
    - 5.2.2 Add batch normalization layers
- 5.3 Lightweight CNN architecture
    - 5.3.1 Simplified CNN with fewer filters (16->8 eg)

1. Deeper CNN with More Conv/Dropout Layers + BatchNorm
- Implements 5.1 and part of 5.2.2
- Adds more convolutional and dropout layers while maintaining batch normalization.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class ExoplanetCNN_Deeper(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.25),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.25),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = ExoplanetCNN_Deeper()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=5e-4)
criterion = nn.BCELoss()

2. Grid Search Candidate with Smaller Kernel Sizes, More BatchNorm, and Tuned Hyperparameters
- Implements 5.2.1 and 5.2.2
- Smaller kernels (kernel_size=2), finer lr, more batch norms, and 

In [ ]:
class ExoplanetCNN_GridSearch(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=2, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# Candidate for Grid Search
model = ExoplanetCNN_GridSearch()
optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-3)
criterion = nn.BCELoss()
batch_size = 64  # Grid search candidate value


3. Lightweight CNN Architecture
- Implements 5.3 and 5.3.1
- Reduces filters for smaller model footprint, ideal for edge/low-power inference.

In [ ]:
class ExoplanetCNN_Lightweight(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = ExoplanetCNN_Lightweight()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.BCELoss()


4. Full Training and Evaluation Pipeline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from torch.utils.data import DataLoader, TensorDataset, random_split

# Dummy dataset generator (replace with your generated synthetic data)
def get_dummy_dataset(num_samples=1000, input_length=128):
    X = torch.randn(num_samples, 1, input_length)
    y = torch.randint(0, 2, (num_samples, 1)).float()
    dataset = TensorDataset(X, y)
    return random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])

# Lightweight CNN class (reused here)
class ExoplanetCNN_Lightweight(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=3, padding=1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(8, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# Training loop
def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            predictions = (outputs.cpu().numpy() > 0.5).astype(int)
            y_pred.extend(predictions)
            y_true.extend(y_batch.numpy())
    
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred)
    }
j

5. Grid Search Integration
- Replace get_dummy_dataset() with your light curve dataset loader.
- You can adjust ExoplanetCNN_Lightweight to ExoplanetCNN_GridSearch or ExoplanetCNN_Deeper as needed.

In [ ]:
# Grid search config
param_grid = {
    'lr': [1e-3, 5e-4],
    'weight_decay': [1e-4, 1e-3],
    'batch_size': [32, 64]
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data, val_data = get_dummy_dataset()
best_score = 0
best_params = None
results = []

for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    
    model = ExoplanetCNN_Lightweight().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    criterion = nn.BCELoss()
    
    train_loader = DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_data, batch_size=128)
    
    for epoch in range(10):
        train_loss = train_model(model, train_loader, optimizer, criterion, device)
    
    metrics = evaluate_model(model, val_loader, device)
    print(f"Metrics: {metrics}")
    
    results.append((params, metrics))
    if metrics['f1'] > best_score:
        best_score = metrics['f1']
        best_params = params

print(f"\nBest Parameters: {best_params}")
